In [35]:
import pymongo
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import string
from slimit import ast
from slimit.parser import Parser
from slimit.visitors import nodevisitor
from slimit import minify

In [36]:
import spacy

In [2]:
# connecting to my mongodb and get data out
client = pymongo.MongoClient('mongodb://localhost:27017/')
# create database
db = client.movie_db
# create colletions
col = db['rotten_tomato']

In [3]:
# get pending movie list fron mongo db
def get_latest_pending_list():
    tomato_data = pd.DataFrame(list(col.find()))
    goal_list = tomato_data.loc[tomato_data['critic(%)'] == 'na'][['name', 'url', 'year']].reset_index()
    
    return goal_list

In [13]:
#search movie in rotten tomato
base_url = 'https://www.rottentomatoes.com/search/?search='
# request url
def get_html(url):
    try:
        r = requests.get(url, timeout=120)
        r.raise_for_status
        #site encoding
        r.encoding = 'utf-8'
        
        return r.text
    except:
        print(url + 'ERROR')
        pass
    
def get_movie_list(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    movie_list = {}
    try:
        movie_list = soup.find_all('script')[32].text
    except:
        movie_list = 'na'
        
    return movie_list

def get_movie_url(data, movie, year):
    text = data
    parser = Parser()
    tree = parser.parse(text)
    # put the node data into a dict
    fields = []
    url_s = 'to find'
    for node in nodevisitor.visit(tree):
        if isinstance(node, ast.Assign):
            try:
                field = {str(node.left.value).replace('"',''): str(node.right.value).replace('"','')}
                fields.append(field)   
            except:
                pass
    # check title and year
    #print(fields)
    for index, field in enumerate(fields):
        try:
            if(field['name'] == movie):
                year_dic = fields[index + 1]
                url = fields[index + 2]
                
                if year_dic['year'] == str(year):
                    url_s = url['url']
                
        except:
            pass
        
    return url_s

In [7]:
# crawl rating data by beautiful soup
def get_content(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    data = {}
    try:
        try:
            data['critic(%)'] = soup.find('span', class_='superPageFontColor').span.text
        except:
            data['critic(%)'] = 'na'
            
        try:
            data['critic_rate'] = soup.find('div', id='scoreStats').findChildren()[0].text[25:].strip()
        except:
            data['critic_rate'] = 'na'
            
        try:
            data['critic_count'] = soup.find('div', id='scoreStats').findChildren()[2].text[18:].strip()
        except:
            data['critic_count'] = 'na'
            
        try:
            data['critic_fresh'] = soup.find('div', id='scoreStats').findChildren()[7].text
        except:
            data['critic_fresh'] = 'na'
        
        try:
            data['critic_rot'] = soup.find('div', id='scoreStats').findChildren()[10].text
        except:
            data['critic_rot'] = 'na'
            
        try:
            data['user(%)'] = soup.find('div', class_='audience-score').find('span', class_='superPageFontColor').text[:2]
        except:
            data['user(%)'] = 'na'
            
        try:
            data['user_rate'] = soup.find('div', class_='audience-info').findChildren()[0].contents[2].strip()
        except:
            data['user_rate'] = 'na'
        
        try:
            data['user_number'] = soup.find('div', class_='audience-info').findChildren()[2].contents[2].strip()
        except:
            data['user_number'] = 'na'

    except:
        print('error')
    
    return data

In [33]:
# check word similarity:
def check_similarity(name1, name2):
    nlp = spacy.load('en')
    check_1 = nlp(name1)
    check_2 = nlp(name2)
    #similarity index
    simi_index = check_1.similarity(check_2)
    print(simi_index)
    return(simi_index)
# blur check function
def similar_check(data, movie, year):
    movie = movie
    year = year
    data = data
    
    parser = Parser()
    tree = parser.parse(data)
    # put the node data into a dict
    fields = []
    url_s = 'to find'
    for node in nodevisitor.visit(tree):
        if isinstance(node, ast.Assign):
            try:
                field = {str(node.left.value).replace('"',''): str(node.right.value).replace('"','')}
                fields.append(field)   
            except:
                pass
    # check title and year
    #print(fields)
    for index, field in enumerate(fields):
        try:
            print(field['name'])
            simi_index = check_similarity(movie, field['name'])
            
            # check if the movie is the one we need
            if(simi_index >= 0.85):
                year_dic = fields[index + 1]
                url = fields[index + 2]
                # check if release year match 
                if year_dic['year'] == str(year):
                    url_s = url['url']
                    print(url_s, simi_index)   
                
        except:
            pass
        
    return url_s

In [22]:
pending_list = get_latest_pending_list()
pending_list

,index,name,url,year
0,6,The Avengers,https://www.rottentomatoes.com/m/Avengers,2012
1,59,It,https://www.rottentomatoes.com/m/it,2017


In [23]:
# search Avengers
avenger_keywords = 'Avengers'
base_movie_page_url = 'https://www.rottentomatoes.com/m/'
avenger_url = base_movie_page_url + avenger_keywords
avenger_data = get_content(avenger_url)

In [24]:
avenger_data

{'critic(%)': '92',
 'critic_count': '341',
 'critic_fresh': '314',
 'critic_rate': '8/10',
 'critic_rot': '27',
 'user(%)': '91',
 'user_number': '1,133,144',
 'user_rate': '4.4/5'}

In [25]:
# update avenger to mongo db
col.find_one_and_update(
        {
            'name': pending_list['name'][0],
            'year': int(pending_list['year'][0])
        },
        {'$set':{
            'url': avenger_url,
            'critic(%)': avenger_data['critic(%)'],
            'critic_rate': avenger_data['critic_rate'],
            'critic_count': avenger_data['critic_count'],
            'critic_fresh': avenger_data['critic_fresh'],
            'critic_rot': avenger_data['critic_rot'],
            'user(%)': avenger_data['user(%)'],
            'user_rate': avenger_data['user_rate'],
            'user_number': avenger_data['user_number']
        }}
    )

{'_id': ObjectId('5c43bee47d76e002594bdc61'),
 'critic(%)': 'na',
 'critic_count': '341',
 'critic_fresh': '314',
 'critic_rate': '8/10',
 'critic_rot': '27',
 'name': 'The Avengers',
 'url': 'https://www.rottentomatoes.com/m/Avengers',
 'user(%)': '91',
 'user_number': '1,133,144',
 'user_rate': '4.4/5',
 'year': 2012}

In [27]:
# search it
final_list = get_latest_pending_list()
final_list

,index,name,url,year
0,59,It,https://www.rottentomatoes.com/m/it,2017


In [28]:
it_keyword = 'it'
it_url = base_url + it_keyword

In [31]:
it_data = get_movie_list(it_url)

In [32]:
it_data

'\n                        require([\'jquery\', \'globals\', \'search-results\', \'bootstrap\'], function($, RT, mount) {\n                            mount($("#search-results-root").get(0), RT.PrivateApiV2FrontendHost, \'it\', {"actorCount":225,"actors":[{"name":"Yuka Itaya","url":"/celebrity/yuka-itaya","image":"https://resizing.flixster.com/LyDwJ2Dcq3jM1N2_1Tjhtm6tsLE=/fit-in/80x80/v1.bjs3NTI4MzY7ajsxNzk2MzsxMjAwOzMwMDs0NTA"},{"name":"Itai Tamir","url":"/celebrity/itai_tamir","image":"https://staticv2-4.rottentomatoes.com/static/images/redesign/poster_default_redesign.gif"},{"name":"Juzo Itami","url":"/celebrity/juzo_itami","image":"https://staticv2-4.rottentomatoes.com/static/images/redesign/poster_default_redesign.gif"},{"name":"Ittoku Kishibe","url":"/celebrity/ittoku_kishibe","image":"https://resizing.flixster.com/kBXVN7qnIpBZimQnNGHeiIy2OEk=/fit-in/80x80/v1.bjszMDU5NjI7ajsxNzk1ODsxMjAwOzMwMDsyMDA"},{"name":"Faking It Three","url":"/celebrity/faking_it_three","image":"https://st

In [37]:
it_url_find = similar_check(it_data, 'It', '2017')
it_url_find

Yuka Itaya
0.18694309388057606
Itai Tamir
0.23591353747208144
Juzo Itami
0.23493180277615083
Ittoku Kishibe
0.24966582555345412
Faking It Three
0.5107553355237401
Itamar Malul
0.1786697076152848
Itai Shcherback
0.24032276735411834
Itziar Atienza
0.14312700941778767
Ita Herbert
0.25336388820421274
Italia Ricci
0.2391116881117965
Hillel Italie
0.20013776170061984
Clara Itzel Hernández Valladares
0.214901993934344
Itrath Syed
0.3002794477932882
Ilana Iturbide
0.1794596094776356
Eric Italiano
0.18826643876926222
Dave Itzkoff
0.27113385736816026
Robert Ito
0.29447288866943083
It
1.0
Clara Bow
0.29164271745754566
Antonio Moreno
0.22527613571210117
William Austin
0.20749722173771487
It
1.0
/m/it_2017 1.0
Bill Skarsgård
0.28176141251054787
Jaeden Lieberher
0.32602719687030624
Jeremy Ray Taylor
0.2648041108450863
I.T.
0.5364551674983981
Pierce Brosnan
0.1997260022641677
Stefanie Scott
0.17098467265311937
Anna Friel
0.2157583056264939
Isn't It Romantic
0.2812797239932239
Rebel Wilson
0.228547492

'/m/it_2017'

In [42]:
# get it data from movie page
it_data_find = get_content('https://www.rottentomatoes.com' + it_url_find)

In [43]:
it_data_find

{'critic(%)': '85',
 'critic_count': '341',
 'critic_fresh': '290',
 'critic_rate': '7.2/10',
 'critic_rot': '51',
 'user(%)': '84',
 'user_number': '64,812',
 'user_rate': '4.1/5'}

In [44]:
# update it data to mongo db
col.find_one_and_update(
        {
            'name': 'It',
            'year': 2017
        },
        {'$set':{
            'url': it_url_find,
            'critic(%)': it_data_find['critic(%)'],
            'critic_rate': it_data_find['critic_rate'],
            'critic_count': it_data_find['critic_count'],
            'critic_fresh': it_data_find['critic_fresh'],
            'critic_rot': it_data_find['critic_rot'],
            'user(%)': it_data_find['user(%)'],
            'user_rate': it_data_find['user_rate'],
            'user_number': it_data_find['user_number']
        }}
    )

{'_id': ObjectId('5c43bee47d76e002594bdc96'),
 'critic(%)': 'na',
 'critic_count': 'na',
 'critic_fresh': 'na',
 'critic_rate': 'na',
 'critic_rot': 'na',
 'name': 'It',
 'url': 'https://www.rottentomatoes.com/m/it',
 'user(%)': 'na',
 'user_number': 'na',
 'user_rate': 'na',
 'year': 2017}